In [92]:
import json
import random
import string
import time
import uuid

In [93]:
# adjust number_of_insurance_quotes = 100, 150, 200, 250 to generate 1000, 1500, 2000, 2500 events, respectively
number_of_insurance_quotes = 250
insurance_quote_duplicates = 2

number_of_policy_updates = 5

deleted_policies_duplicates = 1

customer_id = "rgpp0wkpec"

# Insurance Quote Requests and Policy Events
Generates 1000, 1500, 2000, 2500 events
- 100/150/200/250 x InsuranceQuoteRequest events
- 2 x duplicate InsuranceQuoteRequest events
- 5 x UpdatePolicy events (each policy comes from quote so 100 quotes generated would be 100 policies, etc) 
- 1 x DeletePolicy event
- 1 x duplicate DeletePolicy event


In [94]:
insurance_quote_ids = []
for i in range(number_of_insurance_quotes):
    insurance_quote_ids.append(i+1)

In [95]:
with open('insurance-quote.json', 'r') as file:
    insurance_quote_json = json.loads(file.read())

with open('update.json', 'r') as file:
    policy_updated_json = json.loads(file.read())

with open('delete.json', 'r') as file:
    policy_deleted_json = json.loads(file.read())

    
def jsons_with_id(insurance_quote, policy_updated, policy_deleted, i):
    insurance_quote_request_id = insurance_quote_ids[i]
    new_insurance_quote_request = insurance_quote.copy()

    ts = int(time.time())
    
    new_insurance_quote_request['insuranceQuoteRequestDto']['id'] = insurance_quote_request_id
    new_insurance_quote_request['insuranceQuoteRequestDto']['customerInfo']['customerId'] = customer_id
    new_insurance_quote_request['date'] = ts
    new_insurance_quote_request['insuranceQuoteRequestDto']['date'] = ts

    policy_id =  ''.join(random.choices(string.ascii_lowercase, k=10))

    new_policy_updated = policy_updated.copy()
    new_policy_updated['policy']['policyId'] = policy_id
    new_policy_updated['customer']['customerId'] = customer_id
    new_policy_updated['date'] = ts

    new_policy_deleted = policy_deleted.copy()
    new_policy_deleted['policyId'] = policy_id
    new_policy_deleted['date'] = ts

    return new_insurance_quote_request, new_policy_updated, new_policy_deleted

In [96]:
all_events = []
for i in range(number_of_insurance_quotes):
    insurance_quote, policy_updated, policy_deleted = jsons_with_id(insurance_quote_json, policy_updated_json, policy_deleted_json, i)
    insurance_quotes = [insurance_quote] + [insurance_quote] * insurance_quote_duplicates
    policy_updates = [policy_updated] * number_of_policy_updates
    policy_deletes = [policy_deleted] + [policy_deleted] * deleted_policies_duplicates
    all_events += list(map(lambda x: json.dumps(x), insurance_quotes)) + list(map(lambda x: json.dumps(x), policy_updates)) +  list(map(lambda x: json.dumps(x), policy_deletes))
    
random.shuffle(all_events)

In [97]:
with open('extended-events-2500.txt', 'w') as file:
    for event in all_events:
        file.write(str(uuid.uuid4()))
        file.write(" ")
        event_json = json.loads(event)
        event_id =  event_json['policyId'] if 'kind' in event_json and event_json['kind'] == 'DeletePolicyEvent' else event_json['policy']['policyId'] if 'kind' in event_json and event_json['kind'] == 'UpdatePolicyEvent' else event_json['insuranceQuoteRequestDto']['id']
        file.write(str(event_id))
        file.write(" ")
        file.write(event)
        file.write('\n')

In [98]:
all_events[0]

'{"kind": "UpdatePolicyEvent", "originator": "PolicyManagementBackend", "date": 1661732908, "customer": {"customerId": "rgpp0wkpec", "firstname": "Max", "lastname": "Mustermann", "birthday": 631148400000, "streetAddress": "Oberseestrasse 10", "postalCode": "8640", "city": "Rapperswil", "email": "admin@example.com", "phoneNumber": "055 222 4111", "moveHistory": [], "links": [{"rel": "self", "href": "http://localhost:8110/customers/rgpp0wkpec?fields="}, {"rel": "address.change", "href": "http://localhost:8110/customers/rgpp0wkpec/address"}]}, "policy": {"policyId": "yrqilwcopl", "customer": "rgpp0wkpec", "creationDate": 1618948272375, "policyPeriod": {"startDate": 1618956000000, "endDate": 1619215200000}, "policyType": "Life Insurance", "deductible": {"amount": 100, "currency": "CHF"}, "policyLimit": {"amount": 100000, "currency": "CHF"}, "insurancePremium": {"amount": 1000, "currency": "CHF"}, "insuringAgreement": {"agreementItems": []}, "links": [], "_expandable": ["customer"]}}'